# Compute Fused Dissimilarity Matrix
Compute geodesic fused angle-delay profile (ADP) / timestamp dissimilarity matrix.

In [1]:
from dichasus_cf0x import training_set
import multiprocessing as mp
import scipy.sparse.csgraph
import sklearn.neighbors
import tensorflow as tf
import scipy.spatial
import numpy as np
import tqdm

2023-10-20 14:07:06.542469: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-20 14:07:07.477657: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-10-20 14:07:08.571589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-10-20 14:07:08.596614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [2]:
def csi_time_domain(csi, pos, time):
    csi = tf.signal.fftshift(tf.signal.ifft(tf.signal.fftshift(csi, axes = -1)), axes = -1)
    return csi, pos, time

def cut_out_taps(tap_start, tap_stop):
    def cut_out_taps_func(csi, pos, time):
        return csi[:,:,:,tap_start:tap_stop], pos, time

    return cut_out_taps_func


training_set = training_set.map(csi_time_domain, num_parallel_calls = tf.data.AUTOTUNE)
training_set = training_set.map(cut_out_taps(507, 520), num_parallel_calls = tf.data.AUTOTUNE)

### Load Data

In [3]:
classical_estimated_positions = np.load("results/estimated_positions.npy")

# From TensorFlow to NumPy

In [4]:
groundtruth_positions = []
csi_time_domain = []
timestamps = []

for csi, pos, time in training_set.prefetch(tf.data.AUTOTUNE).batch(1000):
    csi_time_domain.append(csi.numpy())
    groundtruth_positions.append(pos.numpy())
    timestamps.append(time.numpy())

csi_time_domain = np.concatenate(csi_time_domain)
groundtruth_positions = np.concatenate(groundtruth_positions)
timestamps = np.concatenate(timestamps)

# Dissimilarity Metrics and Dissimilarity Matrix Computation

### Step 1: ADP-based dissimilarity matrix

In [5]:
adp_dissimilarity_matrix = np.zeros((csi_time_domain.shape[0], csi_time_domain.shape[0]), dtype=np.float32)

def adp_dissimilarities_worker(todo_queue, output_queue):
    def adp_dissimilarities(index):
        # h has shape (arrays, antenna rows, antenna columns, taps), w has shape (datapoints, arrays, antenna rows, antenna columns, taps)
        h = csi_time_domain[index]
        w = csi_time_domain[index:]

        dotproducts = np.abs(np.einsum("brmt,lbrmt->lbt", np.conj(h), w))**2
        norms = np.real(np.einsum("brmt,brmt->bt", h, np.conj(h)) * np.einsum("lbrmt,lbrmt->lbt", w, np.conj(w)))
        
        return np.sum(1 - dotproducts / norms, axis = (1, 2))

    while True:
        index = todo_queue.get()

        if index == -1:
            output_queue.put((-1, None))
            break
        
        output_queue.put((index, adp_dissimilarities(index)))

with tqdm.tqdm(total = csi_time_domain.shape[0]**2) as bar:
    todo_queue = mp.Queue()
    output_queue = mp.Queue()

    for i in range(csi_time_domain.shape[0]):
        todo_queue.put(i)
    
    for i in range(mp.cpu_count()):
        todo_queue.put(-1)
        p = mp.Process(target = adp_dissimilarities_worker, args = (todo_queue, output_queue))
        p.start()

    finished_processes = 0
    while finished_processes != mp.cpu_count():
        i, d = output_queue.get()

        if i == -1:
            finished_processes = finished_processes + 1
        else:
            adp_dissimilarity_matrix[i,i:] = d
            adp_dissimilarity_matrix[i:,i] = d
            bar.update(2 * len(d) - 1)

 38%|███▊      | 169382464/440874009 [01:04<01:43, 2627547.08it/s]Process Process-15:
Process Process-8:
Process Process-13:
Process Process-10:
Process Process-4:
Process Process-5:
Traceback (most recent call last):
Process Process-3:
Process Process-12:
Process Process-6:
Process Process-16:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()

Traceback (most recent call last):
Process Process-7:
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Process Process-9:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.11/multiprocessing/process.py", line 108, in run

KeyboardInterrupt: 

### Step 2: Timestamp-based dissimilarity matrix

In [ ]:
# Compute timestamp-based dissimilarity matrix
timestamp_dissimilarity_matrix = np.abs(np.subtract.outer(timestamps, timestamps))

### Step 3: Fusing $\mathbf D_\mathrm{ADP}$ with $\mathbf D_\mathrm{time}$

In [ ]:
TIME_THRESHOLD = 2
small_time_dissimilarity_indices = np.logical_and(timestamp_dissimilarity_matrix < TIME_THRESHOLD, timestamp_dissimilarity_matrix > 0)
small_time_dissimilarities = timestamp_dissimilarity_matrix[small_time_dissimilarity_indices]
small_adp_dissimilarities = adp_dissimilarity_matrix[small_time_dissimilarity_indices]

occurences, edges = np.histogram(small_adp_dissimilarities / small_time_dissimilarities, range = (0, 50), bins = 1500)

In [ ]:
bin_centers = edges[:-1] + np.diff(edges) / 2.
max_bin = np.argmax(occurences)
min_threshold = np.quantile(occurences[:max_bin], 0.5)

for threshold_bin in range(max_bin - 1, -1, -1):
    if occurences[threshold_bin] < min_threshold:
        break

scaling_factor = bin_centers[threshold_bin]

In [ ]:
# Fuse ADP-based and time-based dissimilarity matrices
dissimilarity_matrix_fused = np.minimum(adp_dissimilarity_matrix, timestamp_dissimilarity_matrix * scaling_factor)

### Step 4: Geodesic Dissimilarity Matrix

In [ ]:
n_neighbors = 20

nbrs_alg = sklearn.neighbors.NearestNeighbors(n_neighbors = n_neighbors, metric="precomputed", n_jobs = -1)
nbrs = nbrs_alg.fit(dissimilarity_matrix_fused)
nbg = sklearn.neighbors.kneighbors_graph(nbrs, n_neighbors, metric = "precomputed", mode="distance")

In [ ]:
dissimilarity_matrix_geodesic = np.zeros((nbg.shape[0], nbg.shape[1]), dtype = np.float32)

def shortest_path_worker(todo_queue, output_queue):
    while True:
        index = todo_queue.get()

        if index == -1:
            output_queue.put((-1, None))
            break

        d = scipy.sparse.csgraph.dijkstra(nbg, directed=False, indices=index)
        output_queue.put((index, d))

with tqdm.tqdm(total = nbg.shape[0]**2) as bar:
    todo_queue = mp.Queue()
    output_queue = mp.Queue()

    for i in range(nbg.shape[0]):
        todo_queue.put(i)
    
    for i in range(mp.cpu_count()):
        todo_queue.put(-1)
        p = mp.Process(target = shortest_path_worker, args = (todo_queue, output_queue))
        p.start()

    finished_processes = 0
    while finished_processes != mp.cpu_count():
        i, d = output_queue.get()

        if i == -1:
            finished_processes = finished_processes + 1
        else:
            dissimilarity_matrix_geodesic[i,:] = d
            bar.update(len(d))

# Scaling the Dissilimarity Matrix
All values should be in meters...

In [ ]:
scaling_nth_reduction = 30
classical_positions_reduced = classical_estimated_positions[::scaling_nth_reduction]
dissimilarity_matrix_reduced = dissimilarity_matrix_geodesic[::scaling_nth_reduction, ::scaling_nth_reduction]
classical_distance_matrix = scipy.spatial.distance_matrix(classical_positions_reduced, classical_positions_reduced)

In [ ]:
dissimilarity_unit_meters = np.full_like(dissimilarity_matrix_reduced, np.nan)
diff = np.divide(dissimilarity_matrix_reduced, classical_distance_matrix, out = dissimilarity_unit_meters, where = classical_distance_matrix != 0)
dissimilarity_unit_meters = dissimilarity_unit_meters.flatten()
scaling_factor_meters = np.median(dissimilarity_unit_meters[np.isfinite(dissimilarity_unit_meters)])

In [ ]:
dissimilarity_matrix_geodesic_meters = dissimilarity_matrix_geodesic / scaling_factor_meters

In [ ]:
np.save("results/dissimilarity_matrix_geodesic_meters.npy", dissimilarity_matrix_geodesic_meters)